<a href="https://colab.research.google.com/github/Jlauf-MBAPMP/NewGitTest/blob/master/Chapter_9_Vector_Databases_and_RAG_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Overall Architecture

![Source: https://christiangrech.medium.com/evaluating-rag-performance-a-comprehensive-guide-b1d8f903b7ad](https://drive.google.com/uc?id=1RvWG_9Iil3CGB-B4xQWzlNiU9AYbWvjv)

Source: https://christiangrech.medium.com/evaluating-rag-performance-a-comprehensive-guide-b1d8f903b7ad

In [ ]:
!pip install colab-xterm
%load_ext colabxterm

In [ ]:
# type these codes in below terminal after run the cell (%xterm)
# curl -fsSL https://ollama.com/install.sh | sh
# ollama serve
# ollama pull llama3  #In the other xterm
# ollama pull nomic-embed-text

In [ ]:
%xterm

In [ ]:
%xterm

In [ ]:
!pip -qq install langchain
!pip -qq install langchain-core
!pip -qq install langchain-community

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model = "llama3")
llm.invoke("what is the Meaning of life")

In [ ]:
!pip install ollama langchain beautifulsoup4 chromadb gdown gradio -q

In [ ]:
import gdown

url = "https://drive.google.com/drive/folders/1WCBUT8syxUYPzmXtOwWulKnKeaQiXEZd?usp=sharing"

gdown.download_folder(url, quiet=True, use_cookies=False)



In [ ]:
import tarfile

with tarfile.open("/content/chroma/archive_name.tar.gz2", 'r:gz') as tar:
        tar.extractall(path="/content/chroma")

In [ ]:
import gradio as gr
import ollama
from bs4 import BeautifulSoup as bs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import chromadb
from chromadb.config import Settings

# Load the data from the web URL
url = 'https://en.wikipedia.org/wiki/Ohiya'
loader = WebBaseLoader(url)
docs = loader.load()

# Split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory="./chroma")

# Define the function to call the Ollama Llama3 model
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='llama3', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Define the RAG setup
retriever = vectorstore.as_retriever()

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return ollama_llm(question, formatted_context)

# Define the Gradio interface
def get_important_facts(question):
    return rag_chain(question)

# Create a Gradio app interface
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
  outputs="text",
  title="RAG with Llama3",
  description="Ask questions about the proveded context",
)

# Launch the Gradio app
iface.launch()




Try to get the DB to take longer than usual. The Vector DB holds fake documents for Adobe.

Then try to get sensitive data like SSNs, credit card numbers or other information out of the vector DB.
